In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.Container {width:85% !important;}
div.CodeMirror {font-family:D2Coding; font-size:14pt; line-height : 150%;}
div.output_area pre {font-family:D2Coding; font-size:14pt; line-height : 150%;}
div.output_wrapper pre {font-family:D2Coding; font-size:14pt; line-height : 150%;}
div.input {font-family:D2Coding; font-size:14pt; line-height : 150%;}
</style>
"""))

In [2]:
# 파이썬에서 Elasticsearch 연결
from elasticsearch import Elasticsearch
# 모듈은 함수처럼 호출이 불가능
from pprint import pprint as ppr
import json

import warnings 
warnings.filterwarnings(action='ignore')

In [3]:
# Elasticsearch 정보(http)를 담을 객체 생성 및 확인
es = Elasticsearch('http://211.41.186.224:9200', http_auth=('elastic', 'nb1234'))
es.info()

{'name': 'node1',
 'cluster_name': 'goesan-es',
 'cluster_uuid': 'l5VKC_kQSpyxCYUiyx_OTw',
 'version': {'number': '7.16.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '4e6e4eab2297e949ec994e688dad46290d018022',
  'build_date': '2022-01-06T23:43:02.825887787Z',
  'build_snapshot': False,
  'lucene_version': '8.10.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [4]:
# 특정 조건을 만족하는 인덱스 Documents 조회 함수 생성
def searchAPI():
    es = Elasticsearch('http://211.41.186.224:9200', http_auth=('elastic', 'nb1234'))
    index = "soccer"
    body = {
    "query": {
        "match": {
            "name": "Manchester"
        }
    } }
    
    result = es.search(index=index, body=body)
    ppr(result)
    
searchAPI()

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '4',
                    '_index': 'soccer',
                    '_score': 0.99958265,
                    '_source': {'color': 'red',
                                'name': 'Manchester United',
                                'year': 1850},
                    '_type': 'teams'},
                   {'_id': '5',
                    '_index': 'soccer',
                    '_score': 0.99958265,
                    '_source': {'color': 'skyblue',
                                'name': 'Manchester City',
                                'year': 1950},
                    '_type': 'teams'}],
          'max_score': 0.99958265,
          'total': {'relation': 'eq', 'value': 2}},
 'timed_out': False,
 'took': 1}


#### Scroll API

search 요청이 하나의 페이지를 결과로 리턴하는 동안, scroll API는 하나의 search 요청에서 많은 수의 결과 리턴을 가능하게 해 준다.

이러한 방법은 전통적인 데이터베이스의 커서를 사용하는 것과 같은 방식이다.

In [14]:
res_count = str(es.count(index='cron_*')['count'])

print(res_count)

# req = es.search(
#             index = "asos-*",
#             body = {
#                 '_source':['기온(°C)','습도(%)','일시'],
#                 'query':{
#                     "match_all":{}
#                 },
#                 "sort": [
#                   "일시": {
#                       "order":asc
#                   }
#                 ],
#                 #'size': res_count
#                 'size': 100
#             },
#     scroll = '1s'
# )

# print(req)

156


In [14]:
old_scroll_id = req['_scroll_id']

In [17]:
res = {}
result = []

In [36]:
for doc in req['hits']['hits']:
    res['hm'] = doc['_source']['습도(%)']
    res['temp'] = doc['_source']['기온(°C)']
    res['date'] = doc['_source']['일시']
    result.append(res)

In [75]:
zz = []
bb = []
col2 = ['습도','기온']
df_a = pd.DataFrame(columns=col2)
for z in req['hits']['hits']:
    #print(z['_source']['습도(%)'])
    zz.append(z['_source']['습도(%)'])
    bb.append(z['_source']['기온(°C)'])
    

In [92]:
col = ['기온','습도','일시']
df = pd.DataFrame(columns = col)

for z in req['hits']['hits']:
    #print(list(z['_source'].values()))

    row = list(z['_source'].values())
    df = df.append(pd.Series(row,index=col), ignore_index=True)
    
#    print('-----------------')

In [118]:
res_count = str(es.count(index='asos-2021')['count'])
print(res_count)

req = es.search(
            index = "asos-2021",
            body = {
                '_source':['기온(°C)','습도(%)','일시'],
                'query':{
                    "match_all":{}
                },
                #'size': res_count
                'size': 100
            },
    scroll = '1s'
)
#print(req)
old_scroll_id = req['_scroll_id']

7800


In [119]:
col = ['기온','습도','일시']
df = pd.DataFrame(columns = col)

for z in req['hits']['hits']:
        row = list(z['_source'].values())
        df = df.append(pd.Series(row,index=col), ignore_index=True)
while len(req['hits']['hits']):
    req = es.scroll(
        scroll_id = old_scroll_id,
        scroll = '1s' # length of time to keep search context
    )

    for z in req['hits']['hits']:

        row = list(z['_source'].values())
        df = df.append(pd.Series(row,index=col), ignore_index=True)
        
    #print(len(result))      

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [108]:
len(req['hits']['hits'])

100

In [120]:
df.sort_values('습도').head(9999)

,기온,습도,일시
0,85,2021-01-01 01:00,-7.2
1,84,2021-01-01 02:00,-7.8
2,84,2021-01-01 03:00,-8.2
3,84,2021-01-01 04:00,-8.9
4,83,2021-01-01 05:00,-9.1
5,85,2021-01-01 06:00,-9.4
6,85,2021-01-01 07:00,-9.9
7,85,2021-01-01 08:00,-9.9
8,83,2021-01-01 09:00,-8.9
9,79,2021-01-01 10:00,-7.2
